In [ ]:
import re 
import os
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://polkistories.com/collections/earrings"

response = requests.get(url)
response.status_code

In [ ]:
response.status_code 

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
scripts = soup.find_all("script")
print(scripts)

for script in scripts:
    if "collection_viewed" in script.text:
        match = re.search(r'publish\("collection_viewed",\s*(\{.*?\})\s*\)', script.text, re.DOTALL)

        if match:
            collection_data = match.group(1)
            print(collection_data)

            try:
                json_data = json.loads(collection_data)
                with open('earrings_data.json', 'w') as outfile:
                    json.dump(json_data, outfile, indent=4)

                if json_data:
                    data = json_data['collection']['productVariants']
                    print(data)
                    product_url_list = list()
                    for dd in data:
                        product_link = dd['product']['url'] 
                        product_link = "https://polkistories.com" + product_link
                        product_url_list.append(product_link)

            except json.JSONDecodeError as e:
                print("JSON parsing error:", e)

In [ ]:
len(product_url_list)

In [ ]:
# print(product_url_list)

product_data_list = list()
for url in product_url_list:

    product_response = requests.get(url)
    print(url)
    pro_soup = BeautifulSoup(product_response.text, 'html.parser')

    product_url = url
    json_url = url + "/product.json"
    json_response = requests.get(json_url).json()
    product_item_detals = pro_soup.select_one('.prod_item_details')

    # Add your code to extract and save additional product information here
    common_attributes = json_response['product']
    name = common_attributes['title']

    # Images
    sku = common_attributes['variants'][0].get('sku', '')
    images = [image['src'] for image in common_attributes['images']]
    save_dir = "Earrings"
    os.makedirs(save_dir, exist_ok=True)
    for idx, img_url in enumerate(images):
        try:
            response = requests.get(img_url)
            if response.status_code == 200:
                # Make a unique file name
                file_ext = img_url.split('.')[-1].split("?")[0]  # jpg, png, etc.
                file_path = os.path.join(save_dir, f"{sku}_{idx + 1}.{file_ext}")
                
                # Write the image
                with open(file_path, "wb") as f:
                    f.write(response.content)
                
                print(f"✅ Saved: {file_path}")
            else:
                print(f"❌ Failed to download: {img_url}")
        except Exception as e:
            print(f"⚠️ Error downloading {img_url}: {e}")

    # Additional Information
    product_type = common_attributes['product_type']
    brand = common_attributes['vendor']
    item_packge_quantity = pro_soup.select_one('.select-popout__toggle').text
    Gender = "Women"

    # Table
    table = pro_soup.select_one('.prod_item_details')
    metal_purity = table.find("th", string="Metal").find_next_sibling("td").text if table.find("th", string="Metal") else ""
    dimension = table.find("th", string="Dimension").find_next_sibling("td").text if table.find("th", string="Dimension") else ""
    diamond_weight = table.find("th", string="Diamond Weight").find_next_sibling("td").text if table.find("th", string="Diamond Weight") else ""
    diamond_quality = table.find("th", string="Diamond Quality").find_next_sibling("td").text if table.find("th", string="Diamond Quality") else ""
    polki_weight = table.find("th", string="Polki Weight").find_next_sibling("td").text if table.find("th", string="Polki Weight") else ""
    ruby = table.find("th", string="Ruby").find_next_sibling("td").text if table.find("th", string="Ruby") else ""
    emereld = table.find("th", string="Emerald").find_next_sibling("td").text if table.find("th", string="Emerald") else ""
    chain_length = table.find("th", string="Chain Length").find_next_sibling("td").text if table.find("th", string="Chain Length") else ""
    tourmaline = table.find("th", string="Tourmaline").find_next_sibling("td").text if table.find("th", string="Tourmaline") else ""
    floride = table.find("th", string="Floride").find_next_sibling("td").text if table.find("th", string="Floride") else ""
    sapphire = table.find("th", string="Sapphire").find_next_sibling("td").text.replace('\t', '').replace('\n', '').strip if table.find("th", string="Sapphire") else ""
    morganite = table.find("th", string="Morganite").find_next_sibling("td").text if table.find("th", string="Morganite") else ""

    gross_weight = common_attributes['variants'][0].get('weight', '') 
    product_description = BeautifulSoup(common_attributes.get('body_html', ''), 'html.parser').get_text()

    actual_price = common_attributes['variants'][0].get('compare_at_price')
    discount_price = common_attributes['variants'][0].get('price', '')
    print(actual_price, '--', discount_price)
    if actual_price and discount_price:
        discount_percent = ((float(actual_price) - float(discount_price)) / float(actual_price)) * 100

    product_dict = {
        'Product URL': product_url,
        'Product Name': name,
        'Images': images,
        'sku': sku,
        'Product Type': product_type,
        'Brand': brand,
        'Item Packge Quantity': item_packge_quantity,
        'Gender': Gender,
        'Metal Purity': metal_purity,
        'Dimension': dimension,
        'Diamond Weight': diamond_weight,
        'Diamond Quality': diamond_quality,
        'Polki Weight': polki_weight,
        'Ruby': ruby,
        'Emerald': emereld,
        'Chain Length': chain_length,
        'Tourmaline': tourmaline,
        'Floride': floride,
        'Morganite': morganite,
        'Gross Weight': str(gross_weight) + " gm",
        'Product Description': product_description,
        'Actual Price': actual_price,
        'Discount Price': discount_price,
        'Discount Percent': discount_percent
    }

    print(product_dict)
    product_data_list.append(product_dict)

with open('Rings.json', 'w') as outfile:
    json.dump(product_data_list, outfile, indent=4)


In [ ]:
# https://polkistories.com/products/taia-polki-diamond-necklace-ph

In [ ]:
# Merge CSV

import json
 
files = ['bracelates.json', 'earrings.json', 'nacklaces.json', 'pandents.json', 'Rings.json']
 
merged_data = []
 
for f in files:
    with open(f, 'r') as infile:
        data = json.load(infile)
        if isinstance(data, list):
            merged_data.extend(data)
        else:
            merged_data.append(data)
 
with open('all_data.json', 'w') as outfile:
    json.dump(merged_data, outfile, indent=2)

In [ ]:
import json

# Load the JSON file
with open("Final.json", "r") as f:
    data = json.load(f)

# Loop through each item and apply the logic
for item in data:
    actual_price = item.get("Actual Price", "").strip()
    discount_price = item.get("Discount Price", "").strip()

    # If Actual Price is empty but Discount Price is present
    if not actual_price and discount_price:
        item["Actual Price"] = discount_price
        item["Discount Price"] = ""

# Save the updated JSON
with open("product_details.json", "w") as f:
    json.dump(data, f, indent=4)

print("✅ Done! Discount Price shifted to Actual Price wherever needed.")

In [ ]:
import json
import csv

# Load JSON data
with open("product_details.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Dynamically extract all keys + max number of images
all_keys = set()
max_images = 0

for item in data:
    keys = set(item.keys())
    all_keys.update(keys)
    max_images = max(max_images, len(item.get("Images", [])))

# Remove "Images" from keys since we'll manually handle them
all_keys.discard("Images")

# Prepare fieldnames
fieldnames = list(all_keys) + [f"Image_{i+1}" for i in range(max_images)]

# Write to CSV
with open("products_output.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for item in data:
        row = {key: item.get(key, "") for key in all_keys}

        images = item.get("Images", [])
        for i in range(max_images):
            row[f"Image_{i+1}"] = images[i] if i < len(images) else ""

        writer.writerow(row)

print("✅ JSON converted to CSV dynamically, including image columns.")
